In [1]:
import pygmo as pg
import numpy as np
from pybullet_suite import *
import time

pybullet build time: Dec 14 2022 00:46:04


In [2]:
world = BulletWorld(gui=False)
sm = BulletSceneMaker(world)
robot = world.load_robot(Panda, "panda")

argv[0]=


In [3]:
class Prob:
    def __init__(self, xl, xu, q0, fk, jac):
        self.xl = xl
        self.xu = xu
        self.q0 = q0
        self.fk = fk
        self.jac = jac
        self.pg = np.array([0.4, -0.1, 0.3])

    def fitness(self, x):
        obj = 1/2*np.dot(x,x) - np.dot(self.q0, x)
        ce1 = self.fk(x) - self.pg
        return [obj, *ce1]
    
    def get_bounds(self):
        return (self.xl, self.xu)

    def get_nec(self):
        return 3

    def get_nic(self):
        return 0
    
    def gradient(self, x):
        obj_grad = x - self.q0
        ce1_grad = self.jac(x)[:3,:]
        return np.hstack([obj_grad, ce1_grad.flatten()])

In [7]:
prob = pg.problem(Prob(
    robot.arm_lower_limit, 
    robot.arm_upper_limit, 
    robot.arm_central,
    lambda x: robot.forward_kinematics(x).trans,
    lambda x: robot.get_jacobian(x)[:3,:]
))

In [8]:
ip = pg.ipopt()
sqp = pg.nlopt("slsqp")
algo = pg.algorithm(ip)
algo.set_verbosity(200) # in this case this correspond to logs each 200 objevals
pop = pg.population(prob=prob, size=1)
pop.problem.c_tol = [1E-4] * 3

In [9]:
tic = time.time()
algo.evolve(pop)
elapsed = time.time() - tic
print(elapsed)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************


 objevals:        objval:      violated:    viol. norm:
         1         4.0676              3       0.771694 i
0.05949592590332031

Optimisation return status: Solve_Succeeded (value = 0)


0.034281015396118164

In [7]:
pop.champion_x

array([], dtype=float64)

In [20]:
robot.forward_kinematics(pop.get_x()[0,:])

pose: trans[0.400 -0.100 0.300]-rot[0.916 0.270 -0.268 -0.123]]

In [ ]:
uda = pg.nlopt("slsqp")
algo = pg.algorithm(uda)
print(algo) 